# Installing Transformers and importing dependencies

In [1]:
!pip install transformers

In [2]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

# Load Summarization Pipeline

In [3]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to t5-small and revision d769bba (https://huggingface.co/t5-small).
Using a pipeline without specifying a model name and revision in production is not recommended.


C:\Users\DELL\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


# Get a Blog post

In [24]:
URL = "https://www.ibm.com/topics/data-science"

In [25]:
 r = requests.get(URL)

In [26]:
soup = BeautifulSoup(r.text,'html.parser')
results = soup.find_all(['h1','p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [27]:
ARTICLE

'Data science combines math and statistics, specialized programming, advanced analytics, artificial intelligence (AI), and machine learning with specific subject matter expertise to uncover actionable insights hidden in an organization’s data. These insights can be used to guide decision making and strategic planning. The accelerating volume of data sources, and subsequently data, has made data science is one of the fastest growing field across every industry. As a result, it is no surprise that the role of the data scientist was dubbed the “sexiest job of the 21st century” by\xa0Harvard Business Review\xa0(link resides outside of IBM). Organizations are increasingly reliant on them to interpret data and provide actionable recommendations to improve business outcomes. The data science lifecycle involves various roles, tools, and processes, which enables analysts to glean actionable insights. Typically, a data science project undergoes the following stages: Data science is considered a 

# Chunk Text

In [29]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [30]:
sentences = ARTICLE.split('<eos>')

In [33]:
max_chunk = 500
current_chunk = 0
chunks = []

for sentence in sentences:
    if len(chunks) == current_chunk +1:
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

0


In [35]:
for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

# Summarize Text

In [36]:
res = summarizer(chunks, max_length = 120, min_length = 30, do_sample = False)

Token indices sequence length is longer than the specified maximum sequence length for this model (607 > 512). Running this sequence through the model will result in indexing errors
C:\Users\DELL\anaconda3\lib\site-packages\transformers\generation\tf_utils.py:745: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [43]:
summary = ' '.join([summ['summary_text'] for summ in res])
summary

'data science is one of the fastest growing field across every industry . the role of the data scientist was dubbed the “sexiest job of the 21st century” by Harvard Business Review (link resides outside of IBM) data scientists rely on popular programming languages to conduct exploratory data analysis and statistical regression . these open source tools support pre-built statistical modeling, machine learning, and graphics capabilities . multipersona DSML platforms use automation, self-service portals, and low-code/no-code user interfaces . cloud platforms typically have different pricing models, such a per-use or subscriptions, to meet the needs of their end user . when they’re hosted in the cloud, teams don’t need to install, configure, maintain, or update them locally . common use cases include process optimization through intelligent automation and enhanced targeting and personalization to improve the customer experience (CX).'

# Save output to Text file

In [45]:
with open('summary.txt', 'w') as f:
    f.write(summary)